In [ ]:
import pandas as pd
import numpy as np

# 读取 S 类公司指标表（公司为行，指标为列）
df_social = pd.read_excel("ESG_社会类_指标总表_公司为行_完整公司保留.xlsx")

# 自定义议题结构表（议题、指标、属性、权重）
s_meta = pd.DataFrame([
    ["Healthcare Access & Affordability", "Pricing accessibility, affordability risk", "Opportunity", "Management", 7.146666667],
    ["Healthcare Access & Affordability", "Essential product or services access", "", "Exposure", 5.36],
    ["Occupational Health & Safety", "Fatalities", "Risk", "Exposure", 7.146666667],
    ["Occupational Health & Safety", "High-consequence injuries", "", "Exposure", 7.146666667],
    ["Occupational Health & Safety", "Recordable injuries", "", "Exposure", 5.36],
    ["Occupational Health & Safety", "Recordable work-related ill health cases", "", "Exposure", 5.36],
    ["Occupational Health & Safety", "Workplace Safety Management System Coverage", "", "Management", 3.573333333],
    ["Product Safety & Quality", "Maintaining customer trust", "", "Exposure", 7.146666667],
    ["Product Safety & Quality", "Evaluating product governance risk", "", "Management", 5.36]
], columns=["Topic", "Metric", "Risk/Opportunity", "Exposure/Management", "权重"])

# 提取三个议题
topics = s_meta["Topic"].unique()
topic_scores = {}

# 按议题计算
for topic in topics:
    topic_meta = s_meta[s_meta["Topic"] == topic]
    exp_metrics = topic_meta[topic_meta["Exposure/Management"] == "Exposure"]
    mgmt_metrics = topic_meta[topic_meta["Exposure/Management"] == "Management"]
    total_weight = topic_meta["权重"].sum()

    df_topic = df_social[["公司"]].copy()
    
    # 分位数转换为0-10分
    for metric in topic_meta["Metric"]:
        if metric in df_social.columns:
            filled = df_social[metric].fillna(df_social[metric].mean())
            df_topic[metric] = filled.rank(pct=True) * 10
        else:
            df_topic[metric] = np.nan

    # 按公司逐行计算议题得分
    for idx, row in df_topic.iterrows():
        # 敞口加权得分
        exp_score = 0
        exp_total = 0
        for _, m in exp_metrics.iterrows():
            val = row[m["Metric"]]
            if pd.notna(val):
                exp_score += val * m["权重"]
                exp_total += m["权重"]
        exp_score = exp_score / exp_total if exp_total > 0 else np.nan

        # 管理指标得分
        mgmt_vals = []
        mgmt_weights = []
        for _, m in mgmt_metrics.iterrows():
            val = row[m["Metric"]]
            if pd.notna(val):
                mgmt_vals.append(val)
                mgmt_weights.append(m["权重"])
        mgmt_score = np.average(mgmt_vals, weights=mgmt_weights) if mgmt_vals else np.nan

        # 应用议题得分公式
        if topic == "Healthcare Access & Affordability":  # 机遇议题
            if not np.isnan(exp_score) and not np.isnan(mgmt_score):
                score = (0.5 + exp_score / 20) * mgmt_score + (0.5 - exp_score / 20) * 5
            else:
                score = np.nan
        else:  # 风险议题
            if not np.isnan(exp_score) and not np.isnan(mgmt_score):
                score = 7 - (max(exp_score, 2) - mgmt_score)
                score = min(max(score, 0), 10)
            else:
                score = np.nan

        df_topic.at[idx, "议题得分"] = round(score, 2) if pd.notna(score) else np.nan

    topic_scores[topic] = df_topic[["公司", "议题得分"]].rename(columns={"议题得分": f"{topic}_得分"})

# 合并所有议题得分
df_all = topic_scores[topics[0]]
for t in topics[1:]:
    df_all = df_all.merge(topic_scores[t], on="公司", how="outer")

# 总权重比例合成 S_社会总分
weights = {t: s_meta[s_meta["Topic"] == t]["权重"].sum() for t in topics}
total = sum(weights.values())
df_all["S_社会总分"] = sum(
    df_all[f"{t}_得分"] * weights[t] for t in topics
) / total

# 导出
df_all.to_excel("S类_议题打分结果.xlsx", index=False)
